In [2]:
import requests
import json
import pandas as pd
import os
from datetime import datetime
import re
import random 
import time 
from IPython.display import display, clear_output
import glob

In [3]:
def get_event_equipment(event_id):
    """
    Fetches the equipment details (like table and ball sponsor) for a specific event.

    Args:
        event_id (int or str): The ID of the event to fetch data for.

    Returns:
        dict: The JSON response data as a Python dictionary, or None if an error occurs.
    """
    # The URL is an f-string, allowing us to insert the event_id dynamically
    url = f"https://wtt-website-api-prod-3-frontdoor-bddnb2haduafdze9.a01.azurefd.net/api/cms/GetEventEquipmentwithLogo/{event_id}"
    
    # All the headers from your cURL command
    headers = {
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-GB,en;q=0.9,es=q=0.8",
        "cache-control": "no-cache",
        "dnt": "1",
        "origin": "https://www.worldtabletennis.com",
        "pragma": "no-cache",
        "priority": "u=1, i",
        "referer": "https://www.worldtabletennis.com/",
        "sec-ch-ua": "\"Chromium\";v=\"140\", \"Not=A?Brand\";v=\"24\", \"Google Chrome\";v=\"140\"",
        "sec-ch-ua-mobile": "?1",
        "sec-ch-ua-platform": "\"Android\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "secapimkey": "S_WTT_882jjh7basdj91834783mds8j2jsd81",
        "user-agent": "Mozilla/5.0 (Linux; Android 11.0; Surface Duo) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Mobile Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raise an error for bad status codes (4xx or 5xx)
        return response.json()
    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error occurred for event {event_id}: {err}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred for event {event_id}: {err}")
    
    return None

In [90]:
#Search folder with match payloads in in. Obtain all csv files. 
singles_payload_directory = "Data/Processed/Match_payloads"
payload_files = glob.glob(f"{singles_payload_directory}/*.csv")


#Obtain the event_ids from the file names!
successful_event_ids = []

for file in payload_files:    
    payloads_df = pd.read_csv(file)
    event_id = payloads_df["eventId"][0]
    successful_event_ids.append((event_id))  


#Obtain the events shortlist and extract only the succuessful events from successful_event_ids.
events_shortlist = "Data/Processed/Events/shortlist_events.csv"
shortlist_df = pd.read_csv(events_shortlist)
final_events_df = shortlist_df[shortlist_df["EventId"].isin(successful_event_ids)]


#Now add event table and ball sponsors to each event 

enriched_data_list = []
event_total = len(final_events_df)

print(f"---🟢Commencing obtaining event sponsors from {events_shortlist}")

for number, (index, event) in enumerate(final_events_df.iterrows()):
    event_id = event["EventId"]

    match_payloads_file = f"{singles_payload_directory}/{event_id}_singles_payloads.csv"
    match_payloads_df = pd.read_csv(match_payloads_file)
    
    
    mens_matches = match_payloads_df[match_payloads_df["subEventType"] == "Men Singles"]
    num_mens_matches = len(mens_matches)
    womens_matches = match_payloads_df[match_payloads_df["subEventType"] == "Women Singles"]
    num_womens_matches = len(womens_matches)
    total_matches = len(match_payloads_df)

    if total_matches != (num_mens_matches + num_womens_matches):
        print(f"Problem in match count for event {event_id}")  
    
   

    
    
    equipment_json = get_event_equipment(event_id)
    sleep_duration = random.uniform(0.3, 0.8)
    display(f"Obtaining sponsors for event ID:{event_id} 🔄({number+1}/{event_total}) - pausing for {sleep_duration:.1f}s to give the API a break ---")
    clear_output(wait=True)
    time.sleep(sleep_duration)
    
    

    table_sponsor = None
    ball_sponsor = None 
    if equipment_json:
        try: 
            sponsors_df = pd.DataFrame(equipment_json)

            table_sponsor_series = sponsors_df[sponsors_df["sponsorTypeName"] == "Official Table"]["sponsorLink"]
            ball_sponsor_series = sponsors_df[sponsors_df["sponsorTypeName"] == "Official Ball"]["sponsorLink"]
            table_sponsor = table_sponsor_series.iloc[0]
            ball_sponsor = ball_sponsor_series.iloc[0]
            
        except(KeyError,IndexError):
            print(f"    -> Could not parse sponsor data for event {event_id}.")
        
        

    enriched_data_list.append({
    "EventId": event_id,
    "TableSponsorLink": table_sponsor,
    "BallSponsorLink": ball_sponsor,
    "Mens Match Count":num_mens_matches,
    "Womens Match Count": num_womens_matches,
    "Total Match Count": total_matches
    })

print("✅ Completed getting sponsors")
data_to_add_df = pd.DataFrame(enriched_data_list)

enriched_events_df = pd.merge(final_events_df,data_to_add_df)  





✅ Completed getting sponsors


In [104]:
# Manually determined map for sponsors, needs to be manually updated as required. 
cleaned_events_df = enriched_events_df
sponsor_map = {
    'https://www.dhs-sportsglobal.com/': 'DHS',
    'http://www.dhs-sports.com/': 'DHS',
    'https://en.dhs-sports.com/': 'DHS',
    'https://www.doublefish.com/': 'Double Fish',
    'https://www.doublefish.com': 'Double Fish',
    'http://www.yinhe1986.cn/': 'Yinhe',
    'http://www.dhs-sportsglobal.com': 'DHS',
    'https://joola.com/pages/table-tennis?srsltid=AfmBOoquv7qYw0rrAhtEXdZaAsWcFDBkpTNksiGhnOxFif6xCoAajsx_': 'Joola',
    'http://www.yinhe1986.cn': 'Yinhe',
    'https://www.andro.de/en': 'Andro',
    'https://www.stigasports.com/en': 'Stiga',
    'www.doublefish.com': 'Double Fish',
    'https://www.tibhar.com/en/': 'Tibhar',
    'https://butterfly.tt/': 'Butterfly',
    'https://joola.com/': 'Joola',
    'https://www.butterfly-global.com/': 'Butterfly',
    'https://www.stag.in/': 'Stag',
    'https://www.dhs-tt.com/en': 'DHS',
    'www.stigasports.cn': 'Stiga',
    'https://stag.in/': 'Stag',
    'https://www.donic.com/donic/en/': 'Donic',
    'https://tibhar.info/en/': 'Tibhar'
}

cleaned_events_df["BallSponsor"] = cleaned_events_df["BallSponsorLink"].map(sponsor_map)
cleaned_events_df["TableSponsor"] = cleaned_events_df["TableSponsorLink"].map(sponsor_map)


#convert time to datetime object and print in readable form as only date is required. 
cleaned_events_df["StartDate"] = pd.to_datetime(cleaned_events_df["StartDateTime"])
cleaned_events_df["StartDate"] = cleaned_events_df["StartDate"].dt.normalize()


#drop columns I don't need starting with any empty columns

cleaned_events_df.dropna(axis=1, how='all', inplace=True)
cleaned_events_df.reset_index()

columns_to_drop = [ 
    "StartDateTime"
    "EndDateTime",
    "Event_Tier_Name",
    "BallSponsorLink",
    "TableSponsorLink",
    "StartDateTime",
    "EndDateTime",
    "EventTypeId",
    
]

cleaned_events_df.drop(columns = columns_to_drop, inplace = True, errors = 'ignore')

In [105]:
#### Save the final events df with sponsor information
date = datetime.now()
date = date.strftime("%Y%m%d")

output_dir = "Data/Processed/Events"
file_name = f"{output_dir}/{date}_events.csv"



cleaned_events_df.to_csv(file_name,index=False)


In [102]:
sponsor_set = set()


for item in enriched_events_df['BallSponsorLink']:
    sponsor_set.add(item)    


table_sponsors_set = set()
for item in enriched_events_df['TableSponsorLink']:
    sponsor_set.add(item)

sponsor_list = list(sponsor_set)

In [103]:
sponsor_list

['http://www.yinhe1986.cn',
 'https://www.andro.de/en',
 'https://www.stigasports.com/en',
 'http://www.dhs-sportsglobal.com',
 'https://joola.com/pages/table-tennis?srsltid=AfmBOoquv7qYw0rrAhtEXdZaAsWcFDBkpTNksiGhnOxFif6xCoAajsx_',
 'www.doublefish.com',
 'https://www.tibhar.com/en/',
 'http://www.dhs-sportsglobal.com/',
 'https://www.doublefish.com/\t',
 None,
 'https://www.doublefish.com',
 'https://butterfly.tt/',
 'https://joola.com/',
 'https://en.dhs-sports.com/',
 'http://www.yinhe1986.cn/',
 'https://www.butterfly-global.com/',
 'https://www.doublefish.com/',
 'https://www.stag.in/',
 'https://www.dhs-tt.com/en',
 'https://www.dhs-sportsglobal.com/',
 'https://www.dhs-sportsglobal.com/\t',
 'http://www.dhs-sports.com/',
 'www.stigasports.cn',
 'https://stag.in/',
 'https://www.donic.com/donic/en/',
 'https://tibhar.info/en/',
 'http://www.dhs-sports.com']

In [6]:
equipment_json = get_event_equipment(3110)

In [9]:
isinstance(equipment_json,list)

True

In [10]:
a =[]

In [11]:
a== None

False

In [16]:
if not a:
    print(1)

1


In [17]:
equipment_json

[{'eventId': 3110,
  'eventSponsorId': 4672,
  'sponsorTypeId': 15,
  'logo': 'org_logos/9May2023_14_41_20_Ganten 110x55.png_new_0.png',
  'darkLogo': None,
  'sponsorLink': 'https://www.ganten.com.cn/?lang=en',
  'sponsorTypeName': 'Global Premier Partner'},
 {'eventId': 3110,
  'eventSponsorId': 2820,
  'sponsorTypeId': 4,
  'logo': 'org_logos/DHS 110x55px.png',
  'darkLogo': None,
  'sponsorLink': 'https://www.dhs-sportsglobal.com/',
  'sponsorTypeName': 'Official Ball'},
 {'eventId': 3110,
  'eventSponsorId': 4666,
  'sponsorTypeId': 5,
  'logo': 'org_logos/2October2025_19_23_7_donic - White Logo.jpg',
  'darkLogo': 'org_logos/2October2025_19_23_10_Donic black.jpg',
  'sponsorLink': None,
  'sponsorTypeName': 'Official Table'},
 {'eventId': 3110,
  'eventSponsorId': 2850,
  'sponsorTypeId': 17,
  'logo': 'org_logos/14November2024_14_54_1_Li Ning logo website.png',
  'darkLogo': 'org_logos/14November2024_14_53_58_Li Ning Logo-all white.png',
  'sponsorLink': 'www.lining.com',
  'spo